# Exercise 1


## Part I - accessing b-tagging discriminators

First let's import some useful thigs. For this part we will use __FWLite__, the python bindings to the edm CMSSW classes

In [ ]:
import ROOT
import rootpy #hands down, a better version of PyROOT
import rootpy.plotting as plt
import pprint
from DataFormats.FWLite import Events, Handle

Then we load the input file

In [ ]:
events = Events(
    'root://xrootd-cms.infn.it//store/mc/RunIISummer16MiniAODv3/TT_TuneCUETP8M2T4_13TeV-powheg-pythia8/MINIAODSIM/PUMoriond17_94X_mcRun2_asymptotic_v3-v1/00000/0A8930BA-88BE-E811-8BDD-20CF3027A582.root'
)

This is just a python trick to get the first event

In [ ]:
event = events.__iter__().next()

Now, let's create a handle, like you would do in C++. If you use FWLite for your analysis, remember to create the handle __before your event loop!__ Creating a handle is quite slow in python, so better do it only once.

In [ ]:
handle = Handle('vector<pat::Jet>')

Now get the jet collection

In [ ]:
event.getByLabel('slimmedJets', handle)
jets = handle.product()
print jets

Now that we have our vector of jets, we can check the first element and ask its methods.

B-tagging outputs are called _discriminator_, so let's use python's reflection abilities to see what's inside the `pat::Jet` that we can use. Otherwise you can always use [doxygen](http://cmsdoxygen.web.cern.ch/cmsdoxygen/CMSSW_9_2_8/doc/html/d6/d00/classpat_1_1Jet.html).

In [ ]:
[i for i in dir(jets[0]) if 'disc' in i.lower()]

As you can see there are three methods that look interesting. One for adding the discriminators and two to get them, we will use the second one.

In [ ]:
jets[0].getPairDiscri()

As you can see this returns a `std::vector` containing pairs of `std::string` and `float`, corresponding to the discriminator name and its respective value. Let's check all the discriminators available:

In [ ]:
[i.first for i in jets[0].getPairDiscri()]

## Part II - A few comments on event loops

Let's run a simple event loop counting the events:

In [ ]:
counter = 0
for evt in events:
    counter += 1
print counter

OK, pretty standard, right? Now, let's try to do it __again__

In [ ]:
counter = 0
for evt in events:
    counter += 1
print counter

#### What happened?
To save memory the `FWLite.Events` class behaves like a generator, hence once you loop over one entry, that entry is forgotten and you cannot re-run over it, unless you re-load the file completely. Keep that in mind when playing with it!

Let's reload the file, you can always re-execute this cell how many times you need to reset this feature and run on the full set of events

In [ ]:
events = Events(
    'root://xrootd-cms.infn.it//store/mc/RunIISummer16MiniAODv3/TT_TuneCUETP8M2T4_13TeV-powheg-pythia8/MINIAODSIM/PUMoriond17_94X_mcRun2_asymptotic_v3-v1/00000/0A8930BA-88BE-E811-8BDD-20CF3027A582.root'
)

## Part III - plotting the b-tagging discriminator
Now, let's plot few discriminator outputs for jets with $p_T > 20$ GeV and $|\eta| < 2.4$

In [ ]:
histograms = {
    'CSV' : plt.Hist(100,0,1.1),
    'deepCSV' : plt.Hist(100,0,1.1),
}

In [ ]:
####### YOUR CODE HERE! ######
# To fill the histogram
#histograms['CSV'].fill(...)
#Remember, DeepCSV needs both 'pfDeepCSVJetTags:probb' + 'pfDeepCSVJetTags:probbb'

Now, let's plot it! (some cosmetics first, though)

In [ ]:
histograms['CSV'].markercolor = 'blue'
histograms['deepCSV'].markercolor = 'red'
histograms['deepCSV'].xaxis.title = 'DeepCSV'
histograms['deepCSV'].yaxis.title = 'Events'
canvas = plt.Canvas(600, 600)
histograms['CSV'].Draw()
histograms['deepCSV'].Draw('same')
canvas.Draw() #You need this to display inline

## Part IV - FatJet and sub-jet b-tagging

Subjet b tagging is in many respects similar to the b tagging of the "standard" ak4 jets. However, there are some important differences to consider. Since subjets can get close to one another, using a fixed-size track association cone could lead to double-counting of tracks if they are associated to more than one subjet. To avoid such track sharing, the explicit jet-track association is used for subjets where tracks linked to the charged PF jet constituents are associated to the subjet. Similarly, a fixed-size association cone can lead to ambiguities in the secondary vertex and jet flavor assignment for subjets. This problem is resolved by ghost-associating secondary vertices and generator level hadrons/partons to subjets (see [here](https://twiki.cern.ch/twiki/bin/view/CMSPublic/SWGuideBTagMCTools) for more details). An important thing to keep in mind is that the ghost association, in addition to the subjet collection of, also requires collections of original and groomed fat jets to be present since the "ghost" constituents are first associated to fat jets and then to subjets of the groomed fat jets.

A version of fat-jets and sub-jets are already stored in the `miniAOD` event content, clustered with the `AK8` algorithm (anti-$k_T$ with a 0.8 cone). 
  - How many discriminators you see available for the FatJet? Is there any difference from the the ones for AK4?
  - Can you plot the `CSVv2` and `DeepCSV` discriminator output for the Fat-Jets and their sub-jets?
      - __Pro:__ Using `hadronFlavour()` to get the different flavour identification (5: b jet, 4: c jet, 0: light) can you separate the components and plot them in a stacked histogram?

__Hint:__ use [doxygen](http://cmsdoxygen.web.cern.ch/cmsdoxygen/CMSSW_9_2_8/doc/html/d6/d00/classpat_1_1Jet.html) and the interactive nature of python to test yout commands on a single event to test the commands before running the full event loop

In [ ]:
#
# Your code here, this time it will take a while
#